## Neural Networks - Keras

functionality through a structured layers approach:

- Input Layer: Collects sensory information (e.g., hearing, touching).
- Hidden Layers: Two layers process inputs into emotions and feelings.
- Output Layer: Generates decisions or actions based on processed inputs.
This model offers a simplified view of the complex decision-making processes in the human brain.

Most common types of neural networks (neural network trading):

- Perceptron
- Feed forward neural networks
- Multilayer perceptron
- Convolutional neural network
- Recurrent neural network
- Modular neural network

Source:
- https://blog.quantinsti.com/neural-network-python/

### How to train a neural network?

We'll train a neural network to predict stock prices using historical OHLCV (Open, High, Low, Close, Volume) data. Here’s an overview of the process:

1. Training Approaches:
 - Rule-Based: Define rules for outputs based on inputs.
 - Model Training: Adjust weights on a dataset to improve predictions.
2. Dataset Structure:
 - Inputs: OHLCV data.
 - Output: Next day’s Close price (actual y and predicted y').
3. Cost Function:
 - Measures prediction error:
  $$ C = \sum_{}^{}\frac{1}{2}(\widehat{y}-y)^{2} $$
 - Objective: Minimize this cost.
4. Training Process:
 - Compute initial cost with a set of weights.
 - Adjust weights using backpropagation to reduce the cost iteratively.
5. Optimization:
 - Evaluate cost across multiple weight configurations to find the best ones.

By refining weights based on error, the neural network learns to predict stock prices effectively.

In [1]:
pip install yfinance

In [15]:
# Ta-lib is a technical analysis library, which will be used to compute the RSI and Williams %R. 
# These will be used as features in order to train our artificial neural network or ANN.
# Instructions for the Installation: https://blog.quantinsti.com/install-ta-lib-python/
# https://github.com/cgohlke/talib-build/releases -> Install ta_lib-0.6.3-cp313-cp313-win_amd64.whl On Windows win_amd64

import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import random
import talib

In [14]:
random.seed(42)

price_stock = yf.download('AAPL', start='2017-01-01', end='2024-04-09', auto_adjust=True)

[*********************100%***********************]  1 of 1 completed


In [18]:
# Preparing the dataset
# We will be building our input features by using only the OHLC values. 
# This dataset will help us to specify the features for training our neural network in the next step.

price_stock['H-L'] = price_stock['High'] - price_stock['Low']
price_stock['O-C'] = price_stock['Close'] - price_stock['Open']
price_stock['3day MA'] = price_stock['Close'].shift(1).rolling(window=3).mean()
price_stock['10day MA'] = price_stock['Close'].shift(1).rolling(window=10).mean()
price_stock['30day MA'] = price_stock['Close'].shift(1).rolling(window=30).mean()
price_stock['Std_Dev'] = price_stock['Close'].rolling(5).std()
price_stock['RSI'] = talib.RSI(price_stock['Close'].values.flatten(), timeperiod=9)
price_stock['Williams %R'] = talib.WILLR(price_stock['High'].values.flatten(), price_stock['Low'].values.flatten(), price_stock['Close'].values.flatten(), 7)